In [3]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, FollowEvent, ImageSendMessage, StickerSendMessage, \
    StickerMessage, TemplateSendMessage, ButtonsTemplate, MessageAction, ConfirmTemplate, CarouselTemplate, \
    CarouselColumn, PostbackEvent, QuickReply, QuickReplyButton, PostbackAction
)

import json
import requests
import random

app = Flask(__name__)

secretFile = json.load(open("../secretFile.txt","r"))
richmenuId = secretFile['richmenuId']
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']

line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)

def getProfile(userId):

    profile = line_bot_api.get_profile(userId)
    
    print(profile.display_name)
    print(profile.user_id)
    print(profile.picture_url)
    print(profile.status_message)
    
    return profile

def linkRichmenuToUser(userId):
    line_bot_api.link_rich_menu_to_user(userId, richmenuId)

def correctAnswer(answer,trueAnswer):
    if answer == trueAnswer:
        return 'True'
    else:
        return 'False'
    
    
    
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(FollowEvent)
def followMessage(event):
    
    userId = event.source.user_id
    profile = getProfile(userId)
    
    displayName = profile.display_name
    pictureUrl = profile.picture_url
    
    sendMessage = [
        TextSendMessage(
            text="哈囉%s，打開下面選單可以看到我們提供的服務喔!" % (displayName)
        ),
        
        TextSendMessage(
            text="喔對了，這是你醜醜的照片"
        ),
        
        ImageSendMessage(
            original_content_url= pictureUrl,
            preview_image_url= pictureUrl
        ),
        
        StickerSendMessage(
            package_id='11537',
            sticker_id='51626512'
        )
        
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )
    linkRichmenuToUser(userId)

    
@handler.add(MessageEvent,message = StickerMessage)
def stickerMessage(event):
    
    stickerId = str(random.randint(51626494,51626533))
    
    sendMessage = [
        StickerSendMessage(
            package_id='11537',
            sticker_id=stickerId
        )
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )
    
@handler.add(MessageEvent,message = TextMessage)
def textMessage(event):
    
    sendMessage = []

    message = event.message.text
    
    if '[::richmenu::]' in message:
        if '各種語錄' in message:
            
            # 製作button template
            buttons_template_message = TemplateSendMessage(
                alt_text='各種語錄',
                template=ButtonsTemplate(
                    title='各種語錄',
                    text='隨便點一個吧',
                    actions=[
                        MessageAction(
                            label='英雄',
                            text='[::template::]英雄',
                        ),
                        MessageAction(
                            label='科學家',
                            text='[::template::]科學家'
                        ),
                    ]
                )
            )
            
            # 將這個message加到要傳送的訊息裡面
            sendMessage.append(buttons_template_message)
        
        if '古人語錄' in message:
            carousel_template_message = TemplateSendMessage(
                alt_text='Carousel template',
                template=CarouselTemplate(
                    columns=[
                        CarouselColumn(
                            thumbnail_image_url='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/mainCarouselEast.jpg?raw=true',
                            title='東方',
                            text='東方古人',
                            actions=[
                                MessageAction(
                                    label='曾參',
                                    text='[::mainCarousel::][::east::]曾參'
                                ),
                                MessageAction(
                                    label='曹操',
                                    text='[::mainCarousel::][::east::]曹操'
                                )
                            ]
                        ),
                        CarouselColumn(
                            thumbnail_image_url='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/mainCarouselWest.jpg?raw=true',
                            title='西方',
                            text='西方古人',
                            actions=[
                                MessageAction(
                                    label='哥倫布',
                                    text='[::mainCarousel::][::west::]哥倫布'
                                ),
                                MessageAction(
                                    label='阿基米德',
                                    text='[::mainCarousel::][::west::]阿基米德'
                                )
                            ]
                        )
                    ]
                )
            )
            sendMessage.append(carousel_template_message)
        
        if '隨機問答' in message:
            res = requests.get('https://script.google.com/macros/s/AKfycbxofjii4s_Lt29J0QiBB8KJCKoLkZJhnAMkPkoQA5DhDRN2TKQW/exec')
            questionJson = json.loads(res.text)
            
            questionDescription = questionJson['question']
            questionAnswer = questionJson['answer']
            questionOptionsArray = questionJson['options']
            questionOption = '\n'.join(questionOptionsArray)
            
            questionDescription_text_message = TextSendMessage(text=questionDescription)
            questionOption_text_message = TextSendMessage(text=questionOption)
            print(correctAnswer('A',questionAnswer))
            quickReply_text_message = TextSendMessage(text='請選擇',
                               quick_reply=QuickReply(items=[
                                   QuickReplyButton(action=PostbackAction(label="A", text="[::quickReply::]A", data=correctAnswer('A',questionAnswer))),
                                   QuickReplyButton(action=PostbackAction(label="B", text="[::quickReply::]B", data=correctAnswer('B',questionAnswer))),
                                   QuickReplyButton(action=PostbackAction(label="C", text="[::quickReply::]C", data=correctAnswer('C',questionAnswer))),
                                   QuickReplyButton(action=PostbackAction(label="D", text="[::quickReply::]D", data=correctAnswer('D',questionAnswer)))
                               ]))
            sendMessage.append(questionDescription_text_message)
            sendMessage.append(questionOption_text_message)
            sendMessage.append(quickReply_text_message)
            
    if '[::template::]' in message:
        if '英雄' in message:
            
            randNum = str(random.randint(1,5))
            imageUrl='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/templateHero%s.jpg?raw=true' % (randNum)
            
            image_message = ImageSendMessage(
                original_content_url=imageUrl,
                preview_image_url=imageUrl
            )
            
            sendMessage.append(image_message)
        
        if '科學家' in message:
            
            randNum = str(random.randint(1,4))
            imageUrl='https://github.com/raymonddtrt/lineBot_tutorial/blob/master/material/Images/templateScientist%s.jpg?raw=true' % (randNum)
            
            image_message = ImageSendMessage(
                original_content_url=imageUrl,
                preview_image_url=imageUrl
            )
            
            sendMessage.append(image_message)
    
        # 增加確認訊息，詢問用戶是否要離開當前功能
        confirm_template_message = TemplateSendMessage(
            alt_text='Confirm template',
            template=ConfirmTemplate(
                text='要繼續玩嗎?',
                actions=[
                    MessageAction(
                        label='是',
                        text='[::confirmTemplate::][::richmenu::]各種語錄'
                    ),
                    MessageAction(
                        label='否',
                        text='[::confirmTemplate::]離開'
                    )
                ]
            )
        )
    
        sendMessage.append(confirm_template_message)
    
    if '[::confirmTemplate::]離開' in message:
        text_message = TextSendMessage(text='好的，謝謝你的使用')
        
        sendMessage.append(text_message)
        
    
    if '[::mainCarousel::]' in message:
        if '[::east::]' in message:
            if '曹操' in message:
                text_message = TextSendMessage(text='寧可我負天下人，不讓天下人負你')
            else:
                text_message = TextSendMessage(text='吾日三省吾身，四省你')
        if '[::west::]' in message:
            if '哥倫布' in message:
                text_message = TextSendMessage(text='我最偉大的成就不是發現新大陸，是發現你')
            else:
                text_message = TextSendMessage(text='給我一個支點，我能翹動你的心')
        sendMessage.append(text_message)
    
    # 記得判斷 sendMessage 是否為空list
    if sendMessage:
        line_bot_api.reply_message(
            event.reply_token,
            sendMessage
        )

# PostbackEvent順序優於MessageEvent
@handler.add(PostbackEvent)
def postBackMessage(event):

    if event.postback.data == "True":
        text_message = TextSendMessage(text='阿不就好棒棒')
    else:
        text_message = TextSendMessage(text='嫩，這麼簡單也錯')
    print(event.postback.data)
    
    line_bot_api.reply_message(
        event.reply_token,
        text_message
    )
    
if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [28/Apr/2019 16:58:59] "POST / HTTP/1.1" 200 -


簡辰祐
U3e61d72f01f625028c87e434e3a894d0
https://profile.line-scdn.net/0hcm5ZbtGUPEIIIxFPQ8BDFTRmMi9_DToKcEF1dC0gYSctQ3tBMxZydyolY3sgGywVNUJ2IypwZyUg
None


172.17.0.3 - - [28/Apr/2019 16:59:00] "POST / HTTP/1.1" 200 -


玄
U1f39ee75f517c12187fc8a22ebc7edad
https://profile.line-scdn.net/0m04ae69387251d26cd4ae2b5f2534938fbb34e35e12de
None


172.17.0.3 - - [28/Apr/2019 16:59:43] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:05:17] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:05:19] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:05:29] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:05:32] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:05:43] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:05:47] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:05:57] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:06:00] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:06:04] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:06:09] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:10:19] "POST / HTTP/1.1" 200 -


...

172.17.0.3 - - [28/Apr/2019 17:13:14] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:13:15] "POST / HTTP/1.1" 200 -


False


172.17.0.3 - - [28/Apr/2019 17:13:18] "POST / HTTP/1.1" 200 -


False


172.17.0.3 - - [28/Apr/2019 17:13:24] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:13:25] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:13:38] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:13:41] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:13:49] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:13:51] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:13:57] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:14:00] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:14:06] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [28/Apr/2019 17:14:09] "POST / HTTP/1.1" 200 -
